In [44]:
import numpy as np
import os
import scipy.spatial
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA as sklearnPCA
import random
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import warnings
from sklearn import svm
from sklearn.svm import SVC

def readFromFile(path):
    with open(path, 'r') as content_file:
        content = content_file.read()
        return content  # .split(" ")

def writeToFile(array, path, delimiter):
    target = open(path, 'w')

    for line in array:
        target.write(str(line) + delimiter)
    target.close()

def createHistFromPkst(pkts):
    numberOfPkts = 0

    hist = [0] * 1515
    largerThanMTU = 0
    for i in range(0, len(pkts)):
        if pkts[i] not in [0, 40, 52]:
            if pkts[i] < len(hist):
                hist[int(pkts[i])] += 1
                numberOfPkts += 1
            else:
                largerThanMTU += 1
    for h in range(0, len(hist)):
        hist[h] /= float(numberOfPkts)
    if largerThanMTU / float((largerThanMTU + numberOfPkts)) < 0.05:
        return hist
    else:
        return "error"


def plot_features(X, y, labels, nTrain):
    # from sklearn.datasets.samples_generator import make_blobs
    # X, y = make_blobs(n_samples=50, centers=3, n_features=5, random_state=0)
    pca = sklearnPCA(n_components=2)  # 2-dimensional PCA
    transformed = pd.DataFrame(pca.fit_transform(X))
    XPCA = pca.fit_transform(X)

    colors = {0: "red", 1: "blue", 2: "yellow", 3: "pink", 4: "lightgreen"}
    for k in range(len(labels)):
        plt.scatter(transformed[y == k][0], transformed[y == k][1], label=labels[k], c=colors[k], edgecolors=colors[k])
        '''plt.scatter(XPCA[:, 0][nTrain * k:nTrain * (1 + k)],
               XPCA[:, 1][nTrain * k:nTrain * (1 + k)], label=labels[k], c=colors[k],
              edgecolors=colors[k])'''
        # print XPCA[:, 0][nTrain * k:nTrain * (1 + k)]
        # print XPCA[:, 1][nTrain * k:nTrain * (1 + k)]
        # print XPCA[:, 0][nTrain * k:nTrain * (1 + k)]
    # plt.xlim(-1, 1)
    # plt.ylim(-1, 1)
    plt.legend()
    plt.show()

def computerAccuracy(predict_array_bitcoin_test, predict_array_other_test, trueSet):
    accuracy = 0

    for k in range(0, len(predict_array_other_test)):
        if predict_array_other_test[k] not in trueSet:
            accuracy += 1
    for k in range(0, len(predict_array_bitcoin_test)):
        if predict_array_bitcoin_test[k] in trueSet:  # ,2]:
            accuracy += 1

    return accuracy / float(len(predict_array_other_test) + len(predict_array_bitcoin_test))


def computeRecall(predict_array_bitcoin_test, trueSet):
    truePos = 0
    for k in range(0, len(predict_array_bitcoin_test)):
        if predict_array_bitcoin_test[k] in trueSet:
            truePos += 1
    return truePos / float(len(predict_array_bitcoin_test))


def computePrecision(predict_array_other_test, predict_array_bitcoin_test, trueSet):
    truePostive = 0
    falsePositive = 0
    for k in predict_array_bitcoin_test:
        if k in trueSet:
            truePostive += 1

    for k in predict_array_other_test:
        if k in trueSet:
            falsePositive += 1
    precsion = 0
    if float(falsePositive + truePostive) != 0:
        precsion = truePostive / float(falsePositive + truePostive)
    # print truePostive, falsePositive, "true positive, false positive"
    return precsion, (truePostive, falsePositive)

# /home/fatemeh/dataset/Data/SimpleUser/bittorent_noise/vanilla/noise

def buil_db(array, value):
    data_X = []
    data_Y = []
    n = len(array)
    for i in range(0, n):
        data_X.append(array[i])
        data_Y.append(value)

    return data_X, data_Y
def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def convert_stringArrays_to_floatArray(array):
    intArray = []

    for k in array:
        if isfloat(k):
            intArray.append(float(k))
    return intArray


def get_sizeHist_array_1sec(array_hist):
    array_hist[40] = 0
    array_hist[52] = 0
    array_hist[54] = 0
    array_hist[0] = 0
    all_pkt = sum(array_hist)

    for i in range(0, len(array_hist)):
        array_hist[i] /= float(all_pkt)
    return array_hist


def readpktHistDirectFromFile(path, max_read):
    X_data = []
    files = os.listdir(path)
    w = 0
    for f in range(max_read):
        try:
            rnd = random.randrange(0, len(files))
            hist = convert_stringArrays_to_floatArray(readFromFile(path + files[rnd]).split(" "))
            # array_hist = get_sizeHist_array_1sec(array_hist=hist)
            #
            # if len(array_hist) != 0:
            if sum(hist) != 0:
                X_data.append(np.array(hist))
        except:
            w += 1
    # print w
    return X_data


def buildSmallerTrainData(xTrain, y_train, size):
    X = []
    y = []
    seen = []
    while len(seen) < size:
        rand = random.randrange(0, len(xTrain))
        if rand not in seen:
            X.append(xTrain[rand])
            y.append(y_train[rand])
            seen.append(rand)
    return np.array(X), np.array(y)


def read_hist_dataset():
    '''
    In this function, I read the training and testing seperately. train is in histDataset2/train and test is in
    histDataset2/test
    previous dataset: '/home/fatemeh/Bitcoin/dec 28/histDataset2/train/' + n + "/"
    :return:
    '''

    trainin_dataset_X = []
    trainin_dataset_y = []
    bitcoin_test = []
    other_test = []
    names = [ 'httpMultipleTab/2/all', 'httpMultipleTab/3/all', 'httpMultipleTab/1/all',
            
              'noisyBitMultipleTab/2/all', 'noisyBitMultipleTab/3/all',
             'noisyBitMultipleTab/1/all','bitcoin/all']


    trueLablems = [1]
    labels = [0, 0, 0, 1, 1, 1, 1, 1,1]

    i = 0
    for n in names:
        dataset_tr = readpktHistDirectFromFile(path='/home/fatemeh/jan 19/vlm/train/' + n + "/"
                                               , max_read=4000)
        X, y = buil_db(dataset_tr, value=labels[i])
        trainin_dataset_X.extend(X)
        trainin_dataset_y.extend(y)

        dataset_ts = readpktHistDirectFromFile(path='/home/fatemeh/jan 19/vlm/test/' + n + "/", max_read=500
                                               )
        print (n, "####", len(dataset_tr), len(dataset_ts))
        if labels[i] == 0:
            other_test.extend(dataset_ts)
        else:
            bitcoin_test.extend(dataset_ts)
        i += 1
    bitcoin_test = np.array(bitcoin_test)
    other_test = np.array(other_test)
    print ("Finished reading the dataset ...")
    return trainin_dataset_X, trainin_dataset_y, bitcoin_test, other_test





In [47]:
from keras.models import model_from_json
# load json and create model
json_file = open('model5000caida.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model5000caida.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
loaded_model.predict()

Loaded model from disk


In [45]:
trainin_dataset_X, trainin_dataset_y, bitcoin_test, other_test = read_hist_dataset()

httpMultipleTab/2/all #### 4000 500
httpMultipleTab/3/all #### 4000 500
httpMultipleTab/1/all #### 4000 500
noisyBitMultipleTab/2/all #### 4000 500
noisyBitMultipleTab/3/all #### 4000 500
noisyBitMultipleTab/1/all #### 4000 500


FileNotFoundError: [Errno 2] No such file or directory: '/home/fatemeh/jan 19/vlm/train/bitcoin/all/'

In [43]:

number_of_repeat=[100, 50, 25, 10, 10,10, 10]
sizes = [100, 500, 1000, 2000, 5000, 10000]
trueLablems=[1]
i=0
for size in sizes:
    acu_av = 0
    prec_av = 0
    rec_av = 0
    tru_pos, false_pos = 0, 0
    
    for p in range(number_of_repeat[i]):
        trainX, trainY = buildSmallerTrainData(trainin_dataset_X, trainin_dataset_y, size=size)
        clf =SVC(kernel='rbf', C=1000, gamma=0.001)#SVC(kernel='linear', C=1000)
        clf.fit(trainX, trainY)
        bitTest, t = buildSmallerTrainData(bitcoin_test, np.zeros(len(bitcoin_test)), size=1000)
        otherTest, t = buildSmallerTrainData(other_test, np.zeros(len(other_test)), size=1000)
        predict_array_bitcoin_test = clf.predict(bitTest)
        predict_array_other_test = clf.predict(otherTest)
        accuracy = computerAccuracy(predict_array_bitcoin_test, predict_array_other_test, trueSet=trueLablems)
        recall = computeRecall(predict_array_bitcoin_test, trueSet=trueLablems)

        precision, (truePostive, falsePositive) = computePrecision(predict_array_other_test,
                                                                   predict_array_bitcoin_test,
                                                                   trueSet=trueLablems)

        rec_av += recall
        acu_av += accuracy
        prec_av += precision
        tru_pos += truePostive
        false_pos += falsePositive
    
    print ("size of training: ",size)
    print (acu_av / float(number_of_repeat[i]), prec_av / float(number_of_repeat[i]), rec_av / float(number_of_repeat[i]), false_pos / float(
        number_of_repeat[i] * len(otherTest)), tru_pos / float(number_of_repeat[i] * len(bitTest)))
    i += 1

'''
We have up to 4 numbe of open tabs.

size of training:  100
0.7930399999999997 0.8458535192965987 0.7400700000000003 0.15399 0.74007
size of training:  500
0.87886 0.9393219281848867 0.8149200000000002 0.0572 0.81492
size of training:  1000
0.9078800000000001 0.9769051837530914 0.8364800000000002 0.02072 0.83648
size of training:  2000
0.9237 0.9885841820176993 0.8568999999999999 0.0095 0.8569
size of training:  5000
0.9364500000000001 0.9964006546371834 0.8761000000000001 0.0032 0.8761

over tor:

size of training:  100
0.7401800000000002 0.8323586718260485 0.6138 0.13344 0.6138
size of training:  500
0.80372 0.8333940298183468 0.7614200000000001 0.15398 0.76142
size of training:  1000
0.8295800000000001 0.8487272338731685 0.8025199999999999 0.14336 0.80252
size of training:  2000
0.8423999999999999 0.8453020672054585 0.8385 0.1537 0.8385
size of training:  5000
0.85885 0.8499861857056393 0.8717 0.154 0.8717

size of training:  10000
0.86785 0.8535532475341052 0.8882 0.1525 0.8882
'''

size of training:  100
0.50361 0.4733755094997042 0.87775 0.87053 0.87775


KeyboardInterrupt: 